# 0-Tokenizer

Tokenizer（分词器）在 NLP 领域扮演着基础且关键的作用，它将文本分割成单词或子词并转化为数组编号，为模型提供可处理的输入，在文本预处理、语义理解及适配不同语言和任务等方面奠定基础，是连接自然语言文本与计算机可处理数据的重要桥梁.

## 子词分词算法

常见的子词分词算法有三种：

1. 字节对编码（Byte Pair Encoding，BPE）
2. WordPiece
3. Unigram

### BPE

BPE 是一种简单的数据压缩技术，它会迭代地替换序列中最频繁出现的字节对。BPE 依赖一个预分词器，该预分词器会将训练数据分割成单词（在本项目中，我们使用按空格分词的方法作为预分词方法）.

在预分词之后，会创建一组唯一的单词，并确定它们在数据中的出现频率。接下来，BPE 会创建一个基础词表，该词表包含预分词器最初生成的数据中所有唯一单词的符号。然后，会将这对符号从词表中移除，新形成的符号将加入词表。在迭代过程中，BPE 算法会合并频繁出现的符号对.

给定词表的大小，BPE（字节对编码）算法最终会合并出现频率最高的符号对，直到收敛到该大小.

### WordPiece

WordPiece 算法与 BPE 非常相似。WordPiece 首先将词表初始化为包含训练数据中出现的每个字符，然后逐步学习给定数量的合并规则. 与 BPE 不同的是，WordPiece 并不选择最频繁出现的符号对，而是选择那个加入词表后能使训练数据出现的可能性最大化的符号对.

### Unigram

Unigram 算法将其基础词表初始化为大量的符号，然后逐步削减每个符号，以获得一个更小的词表。它会在训练数据上定义一个对数似然损失，以此来确定是否从词表中移除某个符号.

## 训练一个最简单的分词器

在本节中，我们将学习基于 transformers 库来训练你自己的分词器.

### 初始化

首先，我们应该初始化我们的分词器，并确定选择哪种方法。我们将使用字节对编码（BPE）算法.

In [ ]:
from tokenizers import (
    decoders,  # 解码器
    models,  # 模型
    normalizers,  # 正则化器
    pre_tokenizers,  # 预分词器
    processors,  # 处理器
    trainers,  # 训练器
    Tokenizer,  # 分词器类
)

# 初始化一个基于 BPE（字节对编码）模型的分词器
tokenizer = Tokenizer(models.BPE())

# 设置预分词器为 ByteLevel，确保分词器能够处理字节级别的分词
# add_prefix_space=False 表示不在每个单词前添加空格
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

### 定义特殊标记

数据集中存在一些我们不希望被分词的特殊标记，我们会将这些标记定义为特殊标记，并将它们传递给分词器训练器，以防止出现错误的分词情况.

In [ ]:
# 定义特殊标记列表，这些标记在分词过程中需要被特殊处理
special_tokens = ["<|endoftext|>", "<|im_start|>", "<|im_end|>"]

# 初始化 BPE 训练器
trainer = trainers.BpeTrainer(
    vocab_size=6400,  # 设置词汇表大小为 6400
    special_tokens=special_tokens,  # 确保特殊标记被包含在词汇表中
    show_progress=True,  # 显示训练进度
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()  # 初始化字母表为字节级别的字母表
)

### 从文件中读取数据

在本次实验中，我们使用 JSON Lines（jsonl）格式来存储 Tokenizer 训练数据，分词器内置的训练函数要求训练数据以迭代器的形式传入，因此，我们首先获取一个数据读取的生成器.

In [ ]:
import json

def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            yield data['text']

data_path = './toydataset/tokenizer_data.jsonl'

In [5]:
texts = read_texts_from_jsonl(data_path)
print(f'Row 1: {next(texts)}')

Row 1: <|im_start|>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> <|im_start|>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> <|im_start|>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> <|im_start|>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> <|im_start|>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> <|im_start|>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> <|im_start|>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> <|im_start|>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。
文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>


### 开始训练!

我们使用分词器的内置函数 `tokenizer.train_from_iterator` 来训练分词器.

In [ ]:
# 使用分词器的内置函数从迭代器中训练分词器
tokenizer.train_from_iterator(texts, trainer=trainer)

### 设置解码器

In [ ]:
# 设置解码器为 ByteLevel 解码器
tokenizer.decoder = decoders.ByteLevel()

接下来，检查一下特殊标记是否得到了妥善处理。

In [ ]:
# 确保特殊标记的 ID 与预期一致
assert tokenizer.token_to_id("<|endoftext|>") == 0
assert tokenizer.token_to_id("<|im_start|>") == 1
assert tokenizer.token_to_id("<|im_end|>") == 2

### 将训练好的分词器保存到磁盘

In [9]:
import os

tokenizer_dir = "./model/toy_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
tokenizer.model.save(tokenizer_dir) # generate vocab.json & merges.txt

['./model/toy_tokenizer\\vocab.json', './model/toy_tokenizer\\merges.txt']

### 手动创建一份配置文件

In [10]:
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": False,
    "added_tokens_decoder": {
        "0": {
            "content": "<|endoftext|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "1": {
            "content": "<|im_start|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "<|im_end|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "additional_special_tokens": [],
    "bos_token": "<|im_start|>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "<|im_end|>",
    "legacy": True,
    "model_max_length": 32768,
    "pad_token": "<|endoftext|>",
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<|endoftext|>",
    "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{{ '<|im_start|>system\\n' + system_message + '<|im_end|>\\n' }}{% else %}{{ '<|im_start|>system\\nYou are a helpful assistant<|im_end|>\\n' }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\\n' + content + '<|im_end|>\\n<|im_start|>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\\n' }}{% endif %}{% endfor %}"
}

with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


现在我们已经训练了一个简单的分词器，并将其进行保存，接下来，我们试着加载它，并使用其帮助我们对文本进行编解码.

In [ ]:
from transformers import AutoTokenizer

# 从保存的分词器目录加载分词器
tokenizer = AutoTokenizer.from_pretrained("./model/toy_tokenizer")

# 定义一组消息，包括系统消息、用户消息和助手消息
msg = [
    {"role": "system", "content": "你是一个优秀的聊天机器人，总是给我正确的回应！"},
    {"role": "user", "content": '你来自哪里？'},
    {"role": "assistant", "content": '我来自地球'}
]

# 使用分词器的聊天模板功能对消息进行格式化
new_msg = tokenizer.apply_chat_template(
    msg,
    tokenize=False
)

# 打印原始消息和格式化后的消息
print(f'原始文本：{msg}')
print(f'修改文本：{new_msg}')

原始文本：[{'role': 'system', 'content': '你是一个优秀的聊天机器人，总是给我正确的回应！'}, {'role': 'user', 'content': '你来自哪里？'}, {'role': 'assistant', 'content': '我来自地球'}]
修改文本：<|im_start|>system
你是一个优秀的聊天机器人，总是给我正确的回应！<|im_end|>
<|im_start|>user
你来自哪里？<|im_end|>
<|im_start|>assistant
我来自地球<|im_end|>



In [ ]:
# 获取实际词汇表长度（包括特殊符号）
actual_vocab_size = len(tokenizer)
print('tokenizer实际词表长度：', actual_vocab_size)
model_inputs = tokenizer(new_msg)
print('encoder长度：', len(model_inputs['input_ids']))

tokenizer实际词表长度： 1898
encoder长度： 65


In [23]:
model_inputs = tokenizer(new_msg)
print(f'查看分词结果：\n{model_inputs}')

查看分词结果：
{'input_ids': [1, 85, 91, 277, 71, 79, 201, 329, 303, 328, 1528, 265, 683, 1042, 550, 854, 327, 269, 1311, 834, 582, 678, 265, 434, 324, 245, 425, 2, 201, 1, 87, 85, 71, 84, 201, 329, 997, 101, 444, 383, 422, 237, 291, 2, 201, 1, 67, 85, 85, 75, 277, 67, 80, 86, 201, 284, 997, 101, 444, 438, 166, 241, 228, 2, 201], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
input_ids = model_inputs['input_ids']
response = tokenizer.decode(input_ids, skip_special_tokens=False)
print(f'查看解码结果：\n{response}')
print('decoder和原始文本是否一致：', response == new_msg)

查看解码结果：
<|im_start|>system
你是一个优秀的聊天机器人，总是给我正确的回应！<|im_end|>
<|im_start|>user
你来自哪里？<|im_end|>
<|im_start|>assistant
我来自地球<|im_end|>

decoder和原始文本是否一致： True


## 参考资料

- [Hugging Face NLP Course](https://huggingface.co/learn/nlp-course/zh-CN/chapter2/4)